# Sesión 0 — Puente: de Conceptos a Código (ML con Python)

**Autor:** Cesar Garcia  
**Objetivo de la sesión:** conectar las ideas vistas en el curso conceptual (pérdida, gradiente, optimización, batches, generalización) con su equivalente directo en código usando **PyTorch**.

---
## Mapa mental del entrenamiento (recordatorio)

1. **Forward** (predicción)  
2. **Loss** (error)  
3. **Backward** (gradiente)  
4. **Step** (actualización)  
5. Repetir (batches y épocas)

En esta notebook vas a ver ese ciclo literalmente en un *training loop*.


## Concepto → objeto en PyTorch

| Concepto | En PyTorch |
|---|---|
| Datos $x,y$ | `Dataset`, `DataLoader` |
| Modelo (capas) | `torch.nn.Module`, `nn.Linear`, etc. |
| Forward | `y_hat = model(x)` |
| Función de pérdida | `nn.MSELoss()`, `nn.CrossEntropyLoss()` |
| Gradiente | `loss.backward()` |
| Optimización | `torch.optim.Adam(...)` |
| Learning rate | `lr=...` |
| Batch / iteración | `for xb, yb in loader:` |
| Época | `for epoch in range(E):` |
| Regularización | `weight_decay`, `Dropout`, early stopping |
| Evaluación | `model.eval()`, `torch.no_grad()` |


In [ ]:
# (Opcional) Instalación de PyTorch si no está disponible en tu runtime
# En Colab usualmente ya viene instalado. Este bloque verifica e instala solo si hace falta.

import importlib, sys, subprocess

def ensure_torch():
    spec = importlib.util.find_spec("torch")
    if spec is None:
        print("PyTorch no encontrado. Instalando...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "torch", "torchvision", "torchaudio"])
    else:
        print("PyTorch ya está instalado.")

ensure_torch()

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

print("torch version:", torch.__version__)


## Dataset sintético (2 clases)

Vamos a crear un dataset pequeño y fácil de entrenar. La intención no es “ganar accuracy real”, sino **ver el ciclo completo**:
- datos → loader → modelo → loss → backward → optimizer step → evaluación.


In [ ]:
# Datos sintéticos: dos nubes de puntos en 2D
torch.manual_seed(0)

N = 400
x0 = torch.randn(N//2, 2) + torch.tensor([-2.0, 0.0])
x1 = torch.randn(N//2, 2) + torch.tensor([ 2.0, 0.0])

X = torch.cat([x0, x1], dim=0)                  # features
y = torch.cat([torch.zeros(N//2), torch.ones(N//2)]).long()  # labels: 0/1

# DataLoader (batching)
ds = TensorDataset(X, y)
train_loader = DataLoader(ds, batch_size=32, shuffle=True)

X.shape, y.shape, next(iter(train_loader))[0].shape


## Definir un modelo mínimo (MLP)

**Decisión del programador (puente con lo conceptual):**
- Arquitectura: `Linear → ReLU → Linear`
- Salida: **logits** de tamaño 2 (para 2 clases)


In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 2)  # logits para 2 clases
        )

    def forward(self, x):
        return self.net(x)

model = MLP()
model


## Elegir pérdida y optimizador

**Regla práctica clave:**  
`nn.CrossEntropyLoss()` espera **logits** (salida cruda). No apliques `softmax` antes.

- Pérdida: `CrossEntropyLoss`
- Optimizador: `Adam`
- Learning rate: `1e-2` (intencionalmente algo alto para ver progreso rápido en notebook)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

criterion, optimizer


## Training loop (el puente en 10 líneas)

Aquí está el ciclo completo:

- `logits = model(xb)` → **forward**  
- `loss = criterion(logits, yb)` → **loss**  
- `loss.backward()` → **gradiente**  
- `optimizer.step()` → **actualización**  

Nota: `optimizer.zero_grad()` evita acumular gradientes.


In [ ]:
E = 30
for epoch in range(E):
    model.train()
    total_loss = 0.0

    for xb, yb in train_loader:
        optimizer.zero_grad()
        logits = model(xb)              # forward
        loss = criterion(logits, yb)    # loss
        loss.backward()                 # backward
        optimizer.step()                # step

        total_loss += loss.item()

    if (epoch + 1) % 5 == 0:
        print(f"epoch {epoch+1:02d} | loss {total_loss/len(train_loader):.4f}")


## Evaluación mínima: accuracy

**Concepto (Sesión 4):** evaluación en los mismos datos de entrenamiento suele ser optimista.  
Aun así, sirve para verificar que el loop está funcionando.

Buenas prácticas:
- `model.eval()`
- `torch.no_grad()`


In [ ]:
model.eval()
with torch.no_grad():
    logits = model(X)
    pred = logits.argmax(dim=1)
    acc = (pred == y).float().mean().item()

acc


## Generalización (vista previa)

En el curso, lo siguiente será separar `train/val/test` y medir correctamente generalización.

Checklist de “generalización en código”:
- Split de datos
- Métricas en validación
- Regularización si hay overfitting:
  - `weight_decay` (L2)
  - `Dropout`
  - `Early stopping`


In [ ]:
# Ejemplo rápido: cómo aparece L2 (weight decay) en PyTorch
# (No lo usamos aquí, pero así se vería)
adam_l2 = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-4)
adam_l2


## Cierre

Si reconoces este patrón, ya entiendes cómo se ve el aprendizaje en Python:

- Datos → modelo → pérdida → gradiente → actualización  
- Repetición por batches y épocas  
- Evaluación + regularización para generalización

**Siguiente sesión recomendada:** Datos y preprocesamiento (scikit-learn + PyTorch).
